<a href="https://colab.research.google.com/github/100363504/MasterDL/blob/main/Topic_modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

En este código vamos a procesar un conjunto de datos que contiene artículos educativos para conseguir clasificarlos por tópicos y que sea más fácil el manejo de esta amplia base de datos.
Se va a modelar con BERTopic.

In [1]:
# Instalamos e importamos los paquetes necesarios.
!pip install bertopic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.4/143.4 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 12.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 kB 8.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 6.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 42.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 44.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 45.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 70.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━

In [2]:
from bertopic import BERTopic
import re
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
import json
import numpy as np

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [3]:
# Los datos se encuentran almacenados en Drive, por lo que conectamos nuestro entorno a la carpeta donde se encuentran estos datos.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd
file_path = "/content/drive/My Drive/Master UAH/Paralelizacion/data_part1.1.1.1.json"


In [5]:
# Leemos el archivo JSON donde se encuentran los artículos y obtenemos una lista de documentos 'abstract'
docs = []
with open(file_path, "r", encoding="utf-8") as file:
    for line in file:
        data = json.loads(line.strip())
        abstract_text = data['abstract']
        docs.append(abstract_text)

print(docs)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Primero hacemos un preprocesamiento simple de los datos, para ver la diferencia entre un preprocesamiento más sencillo y uno más complejo.

In [ ]:
# Tokenización
docs_tokenized = [nltk.word_tokenize(doc) for doc in docs]

# Normalización de texto: Convertir a minúsculas y eliminar caracteres especiales
docs_preprocessed = [[re.sub(r'[^a-zA-Z]', '', word.lower()) for word in doc] for doc in docs_tokenized]

docs_preprocessed = [' '.join(doc) if isinstance(doc, list) else str(doc) for doc in docs_preprocessed]


Este es el proprocesamiento más complejo. Se usan técnicas como la tokenización, la eliminación de palabras vacías, la normalización de texto y la lematización.

In [6]:
# Tokenización
docs_tokenized = [nltk.word_tokenize(doc) for doc in docs]

# Eliminación de palabras vacías (stop words)
stop_words = set(stopwords.words('english'))
docs_filtered = [[word for word in doc if word.lower() not in stop_words] for doc in docs_tokenized]

# Normalización de texto: Convertir a minúsculas y eliminar caracteres especiales
docs_normalized = [[re.sub(r'[^a-zA-Z0-9]', '', word.lower()) for word in doc] for doc in docs_filtered]

# Lematización
lemmatizer = WordNetLemmatizer()
docs_lemmatized = [[lemmatizer.lemmatize(word) for word in doc] for doc in docs_normalized]

# Unir tokens preprocesados en documentos nuevamente
docs_preprocessed = [' '.join(doc) for doc in docs_lemmatized]



In [7]:
# Por último, creamos y ajustamos el modelo BERTopic en los documentos preprocesados


topic_model = BERTopic()
topics, _ = topic_model.fit_transform(docs_preprocessed)

In [8]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,78567,-1_galaxy_star_group_function,"[galaxy, star, group, function, theory, equati...",[work study vacuum stability classical unstab...
1,0,1458,0_graphene_graphite_bilayer_nanoribbons,"[graphene, graphite, bilayer, nanoribbons, dir...",[using semiclassical approach input experiment...
2,1,1189,1_curve_galois_variety_projective,"[curve, galois, variety, projective, bundle, g...",[study sheaf e smooth projective curve x minim...
3,2,864,2_planet_transit_planetary_transiting,"[planet, transit, planetary, transiting, eccen...",[orbital parameter extrasolar planet significa...
4,3,840,3_pulsar_xray_kev_outburst,"[pulsar, xray, kev, outburst, timing, psr, sou...",[communicate detection soft 20 200 kev gamm...
...,...,...,...,...,...
1030,1029,10,1029_fractional_mittagleffler_reactiondiffusio...,"[fractional, mittagleffler, reactiondiffusion,...",[reaction rate theory inputoutput type model ...
1031,1030,10,1030_qtm_magnet_fe18_tunneling,"[qtm, magnet, fe18, tunneling, giantspin, tota...",[shown resonant quantum tunnelling magnetisati...
1032,1031,10,1031_wannier_ldadmft_pseudopotentials_orbitals,"[wannier, ldadmft, pseudopotentials, orbitals,...",[review formalism selfconsistent gw approximat...
1033,1032,10,1032_etamathrm_tmathrm_engine_carnot,"[etamathrm, tmathrm, engine, carnot, heat, eff...",[study class cyclic brownian heat engine frame...
